<a href="https://colab.research.google.com/github/Hina1008/sample/blob/master/Wordle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
import os
import copy

Result

In [2]:
class WordleResults:
  def __init__(self):
    self.max = []
    self.min = []
    self.ave = []
    self.geans = []

  def set_max(self, max):
    self.max.append(max)

  def get_max(self):
    return self.max

  def set_min(self, min):
    self.min.append(min)

  def get_min(self):
    return self.min

  def set_ave(self, ave):
    self.ave.append(ave)

  def get_ave(self):
    return self.ave

  def set_gean(self, gean):
    self.geans.append(gean)

  def get_gean(self):
    return self.geans

GA

In [3]:
class Gean:
    '''各個体のクラス
        args: 個体の持つ遺伝子情報(np.array)'''
    def __init__(self, value):
        self.value = value
        self.fitness = 0 

    def get_value(self):
      return self.value

    def set_fitness(self, fitness):
        '''個体に対する目的関数(OneMax)の値をself.fitnessに代入'''
        self.fitness = fitness

    def get_fitness(self):
        '''self.fitnessを出力'''
        return self.fitness

In [4]:
class Token():
  def __init__(self):
    self.small_alphabet = [i for i in range(97, 97+26)]
    self.large_alphabet = [i for i in range(65, 65+26)]

  def getAlphabets(self):
    return self.small_alphabet + self.large_alphabet

  """
  ascii ⇨ chr
  """
  def encode(self, ord):
    return chr(ord)

  def decode(self, chr):
    return ord(chr)

In [86]:
from typing_extensions import ParamSpecKwargs
from IPython.lib.security import passwd_check
from enum import Enum

class EvalType(Enum):
    ONLY_PERFECT_MATCH="only_perfect_match"
    INCLUDE_PARTIAL_MATCH="include_partial_match"

class OnlyPerfectMatch:
  def calc(self, target, gean):
    value = 0
    for chr1, chr2 in zip(target, gean):
      if chr1 == chr2:
        value += 1
    return value

class SameWordMatch:

  def calc(self, target, gean):
    value = 0
    for chr in target:
      if chr in gean:
        value += 1
        gean.remove(chr)
    return value/len(target)

class IncludePartialMatch:
  def __init__(self):
    self.only_perfect_match = OnlyPerfectMatch()
    self.same_word_match = SameWordMatch()

  def calc(self, target, chromesome):
    return (self.only_perfect_match.calc(target, chromesome) + self.same_word_match.calc(target, chromesome))/(len(target)+1)


class Wordle():
  def __init__(self, target, parameters, eval=EvalType.ONLY_PERFECT_MATCH):
    self.max_generation = parameters["maxGeneration"]
    self.population = parameters["population"]
    self.crossover_probability = parameters["crossover_probability"]
    self.mutation_probability = parameters["mutation_probability"]

    self.token = Token()

    self.target = [self.token.decode(word) for word in target]
    self.target_length = len(self.target)

    self.parent_geans =[]
    self.child_geans = []
    self.geans = []

    self.fitness_type = eval
    self.calc_class = {
        EvalType.ONLY_PERFECT_MATCH: OnlyPerfectMatch,
        EvalType.INCLUDE_PARTIAL_MATCH: IncludePartialMatch
    }

    self.results = WordleResults()

  """
  初期
  """
  def create_first_generation(self):
    for _ in range(self.population):
      gean = Gean(random.choices(self.token.getAlphabets(), k=self.target_length))
      gean.set_fitness(self._culc_fitness(gean))
      self.parent_geans.append(gean)
    
  # 親選択
  def create_parents_pair(self):
      parents1 = random.sample(self.parent_geans[::2], k=int(self.population/2))
      parents2 = random.sample(self.parent_geans[1::2],k=int(self.population/2))
      return parents1, parents2
  
  # 二点交叉
  def two_point_cross(self, parent1, parent2):
    #交叉点p1, p2
    first_point = random.randint(0, self.target_length-1)
    second_point = random.randint(0, self.target_length-1)

    # 交叉点の小さい方をfirst_pointとする
    if first_point > second_point:
      first_point, second_point = second_point, first_point

    child1 = Gean(parent1[0:first_point] + parent2[first_point:second_point] + parent1[second_point:])
    child1.set_fitness(self._culc_fitness(child1))
    self.child_geans.append(child1)

    child2 = Gean(parent2[0:first_point] + parent1[first_point:second_point] + parent2[second_point:])
    child2.set_fitness(self._culc_fitness(child2))
    self.child_geans.append(child2)

  def mutation(self, parent):
    index = random.randint(0,self.target_length-1)
    print(parent)
    print(index)
    parent[index] = random.choice(self.token.getAlphabets())
    child = Gean(parent)
    child.set_fitness(self._culc_fitness(child))
    self.child_geans.append(child)

  def _selection(self, geans):
    geans_arr = np.array([gean.get_fitness() for gean in geans])
    print("染色体数" + str(len(geans)))
    # print(geans_arr)
    idx = np.random.choice(np.arange(len(geans)), size=self.population, replace=False, p=geans_arr/sum(geans_arr))
    # print(idx)
    # self.child_geans = []
    return idx

  def _culc_fitness(self, gean):
    fitness = self.calc_class[self.fitness_type]().calc(self.target, copy.deepcopy(gean).get_value())
    return fitness

  def run(self):
    self.create_first_generation()
    for _ in range(self.max_generation):
      # 交叉
      parents1, parents2 = self.create_parents_pair()
      for parent1, parent2 in zip(parents1, parents2):
        if random.random() < self.crossover_probability:
          self.two_point_cross(parent1.get_value(), parent2.get_value())
      
      # 突然変異
      for parent in self.parent_geans:
        if random.random() < self.mutation_probability:
          self.mutation(parent.get_value())

      chromesomes = self.parent_geans + self.child_geans

      self.results.set_min(min([gean.get_fitness() for gean in (self.parent_geans + self.child_geans)]))
      self.results.set_max(max([gean.get_fitness() for gean in (self.parent_geans + self.child_geans)]))
      self.results.set_ave(sum([gean.get_fitness() for gean in (self.parent_geans + self.child_geans)])/len((self.parent_geans + self.child_geans)))
      max_index = [gean.get_fitness() for gean in (self.parent_geans + self.child_geans)].index(max([gean.get_fitness() for gean in (self.parent_geans + self.child_geans)]))
      self.results.set_gean((self.parent_geans + self.child_geans)[max_index].get_value())
      # for value in self.results.get_gean()[0]:
      #   print(self.token.encode(value),end="")
      # print("("+ str(len(self.parent_geans)+len(self.child_geans)) +")")
      # print("最大値：" + str(self.results.get_max()[-1]))
      # print("最小値：" + str(self.results.get_min()[-1]))
      # print("平均値：" + str(self.results.get_ave()[-1]))
      
      idx = self._selection(self.parent_geans + self.child_geans)
      print("染色体数" + str(len(chromesomes)))
      print("ルーレット選択" + str(idx))
      self.parent_geans = [chromesomes[index] for index in idx]
      # print(len(self.parent_geans))
      self.child_geans=[]
      # print(len(self.parent_geans))
      for i in idx:
        for value in chromesomes[i].get_value():
          print(self.token.encode(value),end="")
        print("(" + str(chromesomes[i].get_fitness()) + ")")
    # for chromesome in chromesomes:
    #     print(chromesome.get_value())
    #     print(''.join([self.token.encode(gean) for gean in chromesome.get_value()]))
    #     print(chromesome.get_fitness())
    #     print("-----"*30)


In [87]:
# Setting=[Name, MaxGeneration, Population, EliteCount, MaxStagnationGeneration, GenesDimension ...]
settings = {
    "maxGeneration":1000,
    "population":30,
    "crossover_probability": 0.8,
    "mutation_probability": 0.2
}

wordle = Wordle("Gluegent", settings, EvalType.INCLUDE_PARTIAL_MATCH)
wordle.run()

ストリーミング出力は最後の 5000 行に切り捨てられました。
Gluigint(0.75)
Gluctenz(0.6388888888888888)
GluMXent(0.75)
GluBgent(0.875)
GluGgInt(0.75)
GluvfenM(0.75)
GluBgenz(0.75)
GluMgint(0.75)
Gluvfint(0.625)
GluCfenM(0.625)
GFuCfeqH(0.375)
GluCgenW(0.75)
GluGgent(0.875)
Gluxgint(0.75)
GloFment(0.625)
llukgenJ(0.6388888888888888)
GloxtenW(0.5138888888888888)
GlmxgenM(0.625)
[71, 108, 117, 120, 103, 101, 104, 116]
2
[71, 108, 117, 71, 103, 73, 110, 116]
7
[71, 108, 117, 66, 103, 101, 110, 122]
1
[71, 108, 117, 120, 103, 105, 110, 116]
7
[108, 108, 117, 107, 103, 101, 110, 74]
0
[71, 108, 109, 120, 103, 101, 110, 77]
7
染色体数52
染色体数52
ルーレット選択[34 42 38 21 27  6 43  9 49 12 51 45  2 31 25 13  8 30 37  4  0  1 40 32
 36 29 17  7 20 39]
llukgent(0.75)
Gluxgeht(0.75)
GloFment(0.625)
GluCfenM(0.625)
elukgenJ(0.6388888888888888)
Gluxgent(0.875)
GluGgInt(0.75)
Gluvfent(0.75)
GluxginX(0.625)
Gluigint(0.75)
Glmxgeno(0.625)
GluvfenM(0.625)
GluCfent(0.75)
Gluigint(0.75)
GluxginX(0.75)
Gluctenz(0.6388888888888888)
GluCgent(0.87

グラフにプロット

In [28]:
import matplotlib

テスト

In [ ]:
from abc import ABC, ABCMeta, abstractmethod
class AbstractGA(ABC):
    # Setting=[Name, MaxGeneration, Population, EliteCount, MaxStagnationGeneration, GenesDimension ...]
    def __init__(self, parameters):
        self.max_generation = parameters["maxGeneration"]
        self.population = parameters["population"]
        self.crossover_probability = parameters["crossover_probability"]
        self.mutation_probability = parameters["mutation_probability"]
        self.generation =[]
        self.new_generation = []

    # 遺伝子をランダムに作成。一つの個体に対してself.GenesDimensionの数だけ次元を広げる
    def create_init_gene(self):
        genes = list()
        tmp = list()
        for child in range(self.Population):
            for _ in range(self.GenesDimension):
                tmp.append(random.randint(-30,30))
            genes.append(tmp)
            tmp = list()
        # genes.shape=(self.Population, self.GenesDimension)
        return genes



    # 遺伝子の評価
    def evaluate(self, genes, goal):
        # 評価したデータは辞書型で保持。(key, value)=(child_id, fitness)
        evaluated_data = dict()
        for child in range(self.Population):
            # 適応度は高い方がよい（最大で1となるように調整。)
            fitness = 1/(self.eval_func(genes[child], goal) + 1)
            evaluated_data[child] = fitness
        # 評価値に沿って降順ソート
        evaluated_data = dict(sorted(evaluated_data.items(),reverse=True, key=lambda x:x[1]))
        return evaluated_data

    # evaluateの補助関数
    # 目標値との差分を計算
    def eval_func(self, gene, goal):
        sum_value = 0
        # self.GenesDimension空間における原点からのユークリッド距離を計算。
        for id in range(len(gene)):
            sum_value += gene[id]**2
        return abs(np.sqrt(sum_value) - goal)   # goal(目標値)とのずれを計算。


    # genesデータや適応度などを外部ファイルに出力
    def Out_data(self, genes, evaluated_data, generation):
        # 遺伝子の出力(並び替え前), csvと同じカンマ区切り
        gene_path = "./Out/" + str(generation) + ".gene"
        with open(gene_path, "w") as file:
            for child in range(len(genes)):
                for id in range(len(genes[child])):
                    if id==len(genes[child])-1:
                        print(str(genes[child][id]), file=file)
                    else:
                        print(str(genes[child][id]), end=",", file=file)
        # 適応度の出力（並び替え後）,csvと同じカンマ区切り
        fitness_path = "./Out/" + str(generation) + ".fit"
        with open(fitness_path, "w") as file:
            for id, fitness in evaluated_data.items():
                print(str(id) +","+ str(fitness), file=file)


    # 適応度の高い上位個体を保存
    def Save_elite(self, genes, evaluated_data):
        elite_id = list()
        elite_genes = list()
        # elite_idを辞書から取得
        for key in evaluated_data.keys():
            elite_id.append(key)
        # eliteの数だけ上から抽出
        for elite in range(self.EliteCount):
            elite_genes.append(genes[elite_id[elite]])

        return elite_genes




    # method==0: Expected value selection
    # method==1: Ranking selection
    # method==2: Tournament selection
    def select(self, method, genes, evaluated_data):
        # idとfitnessを個別にリストで保管
        id_list = list()
        fitness_list = list()
        Select_id = list()
        for id, fitness in evaluated_data.items():
            id_list.append(id)
            fitness_list.append(fitness)

        # 期待値選択
        if method==0:
            roulette = 360
            tmp_list = list()
            sum_fitness = sum(fitness_list) # 適応度の合計値を取得
            cumulative_fitness = np.cumsum(roulette*np.array(fitness_list)/sum_fitness) # 適応度の高さに応じて幅の広い一回転360度のテーブルを作成
            for child in range(self.Population - self.EliteCount):    # エリート以外の個体を作成するため、個体数 ー エリート個体数
                for _ in range(2):                                    # ２つの遺伝子を組み合わせるため繰り返す。
                    Fate_dice = random.uniform(0,360)                 # 0~360の間の運命のサイコロを振る
                    Hit_id = 0
                    while True:
                        if cumulative_fitness[Hit_id] >= Fate_dice:
                            break
                        else:
                            Hit_id += 1
                    tmp_list.append(id_list[Hit_id])
                Select_id.append(tmp_list)
                tmp_list = list()
        return Select_id



    # method==0: Uniform crossover
    # method==1: Multipoint crossover
    # method==2: Partial coincidence crossover
    def crossover(self, method, genes, Select_id):
        new_genes = list()
        # 次の世代のgenesをSelect_id[child][0]のid番号を使って作成
        for child in range(self.Population - self.EliteCount):
            new_genes.append(genes[Select_id[child][0]])

        if method==0:
            probability = 0.4
            for child in range(self.Population - self.EliteCount):
                for dim in range(len(genes[0])):
                    Fate_random = random.uniform(0,1)
                    if Fate_random <= probability:
                        new_genes[child][dim] = genes[Select_id[child][1]][dim]
        return new_genes



    def mutation(self, genes, Stagnation):
        probability = 0.4                          #突然変異をする確率
        serious_rate = Stagnation/(self.MaxGeneration - self.EliteCount)   # 停滞が大きくなれば突然変異する可能性も上げる
        serious_count = int(len(genes)*serious_rate)        # genes配列の大きさを掛けることで、配列のどこまでを突然変異与えるのか調整
        for child in range(serious_count):
            for dim in range(len(genes[0])):
                Fate_random = random.uniform(0,1)      # 0~1の乱数を取得
                if Fate_random <= probability:
                    genes[child][dim] += random.randint(-10,10) # -10~10の乱数を付与（決め打ち）
        return genes

## サンプル

In [ ]:
class GA:
    # Setting=[Name, MaxGeneration, Population, EliteCount, MaxStagnationGeneration, GenesDimension ...]
    def __init__(self, Setting):
        self.Name = Setting[0]
        self.MaxGeneration = Setting[1]
        self.Population = Setting[2]
        self.EliteCount = Setting[3]
        self.MaxStagnationGeneration = Setting[4]
        self.GenesDimension = Setting[5]

        os.makedirs("./Out/", exist_ok=True)

        print("\n GA start!! \n")


        # flag==0、print出力 flag==1、ファイル記述
    def get_parameter(self, flag=0, out_path="./"):
        # ファイル出力の際のパス
        out_path = out_path + "GA_parameter.txt"
        # 出力内容
        contents = [
                     f'GA_parameter!!\n',
                     f'Name: {self.Name}',
                     f'MaxGeneration: {self.MaxGeneration}',
                     f'Population: {self.Population}',
                     f'EliteCount: {self.EliteCount}',
                     f'GenesDimension: {self.GenesDimension}',
                                                                ]
        # flagにより出力先を変更
        if flag==0:
            for sentense in contents:
                print(sentense)
        elif flag==1:
            with open(out_path, mode="w") as file:
                for sentense in contents:
                    print(sentense, file=file)


    # GAを始める関数（というか、この中に処理ほとんど書いてるからmain関数みたいなもの？）
    def Start_GA(self):
        # 現在の世代
        generation = 0
        # 世代の停滞値
        Stagnation = 0
        # 目標値
        goal = 0
        # 一番適応度の良い遺伝子の軌跡
        top_genes = list()
        # 初期集団として遺伝子をランダムに作成。
        genes = self.make_genes()
        while(True):
            #　最大世代まで進むと終了
            if generation >= self.MaxGeneration:
                break
            else:
                # 適応度の評価
                evaluated_data = self.evaluate(genes, goal)
                # 遺伝子ファイルと適応度を外部に出力
                self.Out_data(genes, evaluated_data, generation)
                # 適応度の高い上位個体を保存
                elite_genes = copy.deepcopy(self.Save_elite(genes, evaluated_data))
                # 適応度の高い者同士を選択
                Select_id = self.select(0, genes, evaluated_data)
                # 交叉させて新しい遺伝子の作成
                genes = self.crossover(0, genes, Select_id)
                # 突然変異を与えて解探索領域を広げる
                genes = self.mutation(genes, Stagnation)
                # エリート遺伝子を加える
                genes[len(genes):len(genes)] = elite_genes
                # 一番適応度の良い個体を保存
                top_genes.append(elite_genes[0])
                # 第二世代以降において、適応度の停滞（適応度の一番良い値が更新されない）が最大停滞数を超えていればプログラムは終了。
                if len(top_genes)>=2:
                    if top_genes[-1]==top_genes[-2]:
                        # 最大停滞数を超えればアルゴリズムは終了
                        if Stagnation >= self.MaxStagnationGeneration:
                            exit()
                        else:
                            Stagnation += 1
                    else:
                        Stagnation = 0
                # 世代を進める
                generation +=1



    # 遺伝子をランダムに作成。一つの個体に対してself.GenesDimensionの数だけ次元を広げる
    def make_genes(self):
        genes = list()
        tmp = list()
        for child in range(self.Population):
            for _ in range(self.GenesDimension):
                tmp.append(random.randint(-30,30))
            genes.append(tmp)
            tmp = list()
        # genes.shape=(self.Population, self.GenesDimension)
        return genes



    # 遺伝子の評価
    def evaluate(self, genes, goal):
        # 評価したデータは辞書型で保持。(key, value)=(child_id, fitness)
        evaluated_data = dict()
        for child in range(self.Population):
            # 適応度は高い方がよい（最大で1となるように調整。)
            fitness = 1/(self.eval_func(genes[child], goal) + 1)
            evaluated_data[child] = fitness
        # 評価値に沿って降順ソート
        evaluated_data = dict(sorted(evaluated_data.items(),reverse=True, key=lambda x:x[1]))
        return evaluated_data

    # evaluateの補助関数
    # 目標値との差分を計算
    def eval_func(self, gene, goal):
        sum_value = 0
        # self.GenesDimension空間における原点からのユークリッド距離を計算。
        for id in range(len(gene)):
            sum_value += gene[id]**2
        return abs(np.sqrt(sum_value) - goal)   # goal(目標値)とのずれを計算。


    # genesデータや適応度などを外部ファイルに出力
    def Out_data(self, genes, evaluated_data, generation):
        # 遺伝子の出力(並び替え前), csvと同じカンマ区切り
        gene_path = "./Out/" + str(generation) + ".gene"
        with open(gene_path, "w") as file:
            for child in range(len(genes)):
                for id in range(len(genes[child])):
                    if id==len(genes[child])-1:
                        print(str(genes[child][id]), file=file)
                    else:
                        print(str(genes[child][id]), end=",", file=file)
        # 適応度の出力（並び替え後）,csvと同じカンマ区切り
        fitness_path = "./Out/" + str(generation) + ".fit"
        with open(fitness_path, "w") as file:
            for id, fitness in evaluated_data.items():
                print(str(id) +","+ str(fitness), file=file)


    # 適応度の高い上位個体を保存
    def Save_elite(self, genes, evaluated_data):
        elite_id = list()
        elite_genes = list()
        # elite_idを辞書から取得
        for key in evaluated_data.keys():
            elite_id.append(key)
        # eliteの数だけ上から抽出
        for elite in range(self.EliteCount):
            elite_genes.append(genes[elite_id[elite]])

        return elite_genes




    # method==0: Expected value selection
    # method==1: Ranking selection
    # method==2: Tournament selection
    def select(self, method, genes, evaluated_data):
        # idとfitnessを個別にリストで保管
        id_list = list()
        fitness_list = list()
        Select_id = list()
        for id, fitness in evaluated_data.items():
            id_list.append(id)
            fitness_list.append(fitness)

        # 期待値選択
        if method==0:
            roulette = 360
            tmp_list = list()
            sum_fitness = sum(fitness_list) # 適応度の合計値を取得
            cumulative_fitness = np.cumsum(roulette*np.array(fitness_list)/sum_fitness) # 適応度の高さに応じて幅の広い一回転360度のテーブルを作成
            for child in range(self.Population - self.EliteCount):    # エリート以外の個体を作成するため、個体数 ー エリート個体数
                for _ in range(2):                                    # ２つの遺伝子を組み合わせるため繰り返す。
                    Fate_dice = random.uniform(0,360)                 # 0~360の間の運命のサイコロを振る
                    Hit_id = 0
                    while True:
                        if cumulative_fitness[Hit_id] >= Fate_dice:
                            break
                        else:
                            Hit_id += 1
                    tmp_list.append(id_list[Hit_id])
                Select_id.append(tmp_list)
                tmp_list = list()
        return Select_id



    # method==0: Uniform crossover
    # method==1: Multipoint crossover
    # method==2: Partial coincidence crossover
    def crossover(self, method, genes, Select_id):
        new_genes = list()
        # 次の世代のgenesをSelect_id[child][0]のid番号を使って作成
        for child in range(self.Population - self.EliteCount):
            new_genes.append(genes[Select_id[child][0]])

        if method==0:
            probability = 0.4
            for child in range(self.Population - self.EliteCount):
                for dim in range(len(genes[0])):
                    Fate_random = random.uniform(0,1)
                    if Fate_random <= probability:
                        new_genes[child][dim] = genes[Select_id[child][1]][dim]
        return new_genes



    def mutation(self, genes, Stagnation):
        probability = 0.4                          #突然変異をする確率
        serious_rate = Stagnation/(self.MaxGeneration - self.EliteCount)   # 停滞が大きくなれば突然変異する可能性も上げる
        serious_count = int(len(genes)*serious_rate)        # genes配列の大きさを掛けることで、配列のどこまでを突然変異与えるのか調整
        for child in range(serious_count):
            for dim in range(len(genes[0])):
                Fate_random = random.uniform(0,1)      # 0~1の乱数を取得
                if Fate_random <= probability:
                    genes[child][dim] += random.randint(-10,10) # -10~10の乱数を付与（決め打ち）
        return genes